# Optimisation

By combining veriaous concepts like:
* Batch
* Multithreading
* ainvoke

We can optimize our workflow. 

In [8]:
import os
import concurrent.futures
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()
# Step 1: Define the prompt template
prompt = PromptTemplate.from_template("Translate '{text}' to French.")

# Step 2: Initialize the LLM
llm = llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    openai_api_type="azure",
    api_version="2024-12-01-preview",
    azure_endpoint="https://i4talent-openai.openai.azure.com",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0
)

# Step 3: Prepare batch inputs
texts = ["Hello", "Good morning", "How are you?"]
formatted_prompts = [prompt.format(text=t) for t in texts]

# Step 4: Run batch processing
results = llm.batch(formatted_prompts)

# Step 5: Print results
for i, res in enumerate(results):
    print(f"Input: {texts[i]} → Output: {res}")


Input: Hello → Output: content="'Hello' in French is **Bonjour**." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CQrXn0d9qqPUWGbiWupqW9ZEoMVvi', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity':

In [9]:
import asyncio
import nest_asyncio
nest_asyncio.apply()


async def async_call(text):
    prompt_text = prompt.format(text=text)
    return await llm.ainvoke(prompt_text)

async def run_parallel(texts):
    tasks = [async_call(t) for t in texts]
    return await asyncio.gather(*tasks)

results = asyncio.run(run_parallel(texts))
for i, res in enumerate(results):
    print(f"Input: {texts[i]} → Output: {res.content}")



Input: Hello → Output: 'Hello' in French is **Bonjour**.
Input: Good morning → Output: 'Good morning' in French is **Bonjour**.
Input: How are you? → Output: 'How are you?' in French is **"Comment ça va ?"**


In [10]:
def llm_generate(llm, prompt_text):
    return llm.invoke(prompt_text)

prompts = {i: prompt.format(text=t) for i, t in enumerate(texts)}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {
        key: executor.submit(llm_generate, llm, prompt)
        for key, prompt in prompts.items()
    }
    results = {key: future.result() for key, future in futures.items()}

for key, output in results.items():
    print(f"Input: {texts[key]} → Output: {output}")


Input: Hello → Output: content="'Hello' in French is **Bonjour**." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CQrXogXhZfAAKk8tTi0MKqnTGYLsJ', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity':